In [33]:
from langchain.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers.list import ListOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain_core.output_parsers import StrOutputParser , CommaSeparatedListOutputParser , ListOutputParser
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
GOOGLE_API_KEY = os.getenv("GEMINI_API_KEY")

# Initialize the Gemini model
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=GOOGLE_API_KEY)

C:\Users\DELL\AppData\Roaming\Python\Python312\site-packages\IPython\core\interactiveshell.py:3579: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
with open('../test_html.html', 'r') as f:
    html = f.read()

# print(html)

In [3]:
# Create prompt template
prompt =ChatPromptTemplate.from_template( """
**Act as an expert HTML developer and content strategist.** Your role is to generate a precise and comprehensive list of key questions that a user should answer in order to fully customize the **text content** of an HTML template.  

### **Instructions:**  
1. **Analyze the provided HTML template:**  
   - Identify all text-based elements, including headings, paragraphs, buttons, navigation links, form placeholders, and any other textual content.  
   - Exclude non-text elements such as images, colors, fonts, and layout settings.  

2. **Generate key questions:**  
   - Focus only on **text-related customization** (e.g., wording, tone, branding consistency, and localization).  
   - Ensure the questions guide the user in making informed choices about the **tone, clarity, and structure** of the content.  

3. **Organize the questions logically:**  
   - Begin with general content structure (e.g., page headings and main text) and move to specific elements (e.g., button labels and form placeholders).  
   - Cover both **static text** (like headings and paragraphs) and **interactive text** (like form labels and call-to-action buttons).  

4. **Limit the output to exactly 10 questions.**  

### **HTML Template:**  
```html
{html_template}
```  

### **Output Format:**  
- Provide the questions in a **single-line, comma-separated format**, without any additional text.  
- Example output:  
  ```
  1. What should be the main heading of the page?, 2. What introductory text should appear in the hero section?, 3. What should the call-to-action button say?, ...etc
  ```

IMPORTANT: Do not use commas (,) inside your questions, as I am using CommaSeparatedListOutputParser. Instead, use a dash (-) to separate elements within a question.

**Take a deep breath and work on this problem step by step.**  
"""
)



In [4]:
# prompt = PromptTemplate(
#     template=template,
#     input_variables=["html_template"]

# )


In [5]:
parser = CommaSeparatedListOutputParser()
chain = prompt | llm  | parser

In [6]:
response = chain.invoke({
    'html_template' : html
    })  # Invoke with a string

In [7]:
print(response)

['1. What is the desired brand name- logo text for the website header and footer?', '2. What wording should be used for the main heading (H1) on the hero section of the landing page?', '3. What descriptive text should accompany the main heading in the hero section to explain the business offering?', '4. What call-to-action text should be used on the primary button in the hero section?', '5. What heading should be used for the "Features" section to best describe the benefits offered?', '6. What wording should be used for each of the individual feature titles- for example- Smart Automation- Advanced Analytics- Enterprise Security?', '7. What wording should be used for the "Testimonials" section heading to convey customer satisfaction?', '8. What is the desired wording for the call-to-action heading and supporting text in the email collection section?', '9. What placeholder text should be used in the email input field of the email collection form?', '10. What wording should be used for th

In [8]:
for i in  response:    
    print(i)

1. What is the desired brand name- logo text for the website header and footer?
2. What wording should be used for the main heading (H1) on the hero section of the landing page?
3. What descriptive text should accompany the main heading in the hero section to explain the business offering?
4. What call-to-action text should be used on the primary button in the hero section?
5. What heading should be used for the "Features" section to best describe the benefits offered?
6. What wording should be used for each of the individual feature titles- for example- Smart Automation- Advanced Analytics- Enterprise Security?
7. What wording should be used for the "Testimonials" section heading to convey customer satisfaction?
8. What is the desired wording for the call-to-action heading and supporting text in the email collection section?
9. What placeholder text should be used in the email input field of the email collection form?
10. What wording should be used for the navigation links (e.g.- Fea

# Html customiz chat bot

In [59]:
# Define your desired data structure.
class HtmlTemplite(BaseModel):
    html_template: str = Field(description="Html template with updated content after customization") 
# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=HtmlTemplite)

In [52]:
# Create prompt template
prompt =ChatPromptTemplate.from_template( """
**Act as an expert HTML developer and content strategist.**  
You have extensive experience in web development and content customization, specializing in dynamically adapting HTML templates based on user preferences.  

### **Objective:**  
Your task is to analyze a given HTML template and modify its **text content** based on the responses provided by the user. The goal is to ensure that the final HTML output accurately reflects the user's preferences while maintaining structural integrity and proper formatting.  

### **Process:**  
1. **Analyze User Inputs:**  
   - Carefully examine the provided **questions and user responses** to understand their content preferences.  
   - Identify key phrases, themes, and context to determine what changes need to be applied to the template.  

2. **Modify the HTML Template:**  
   - Update **only the text content** within the template while preserving the existing HTML structure, including elements, attributes, and styles.  
   - Ensure that text replacements maintain coherence and proper alignment with the template's purpose.  
   - If any user input is unclear or missing, generate a reasonable default response that aligns with the context.  

3. **Ensure Readability & Formatting:**  
   - Maintain proper text spacing, line breaks, and formatting conventions.  
   - If necessary, adjust the HTML markup to improve readability without altering functionality.  

4. **Output the Updated HTML Template:**  
   - Provide the final modified HTML code while keeping the structure intact.  
   - Ensure that all user preferences are accurately reflected in the text content.  

### **Inputs:**  
#### **HTML Template:**  
```html  
{html_template}  
```  

#### **Questions and User Responses:**  
{Q_A_list}  

### **Expected Output:**  
A fully updated HTML template with text content modified according to the user's answers, ensuring clarity, relevance, and consistency.  
Your html code should be start with body <body> tag and end with body tag </body>.
please answer in json like below:
```json
{{
  "html_template": "<body>Updated HTML code here...</body>"
}}
```

{format_instructions}

Take a deep breath and work on this problem step by step.  


""",
partial_variables={"format_instructions": parser.get_format_instructions()}
)



In [53]:
parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"html_template": {"title": "Html Template", "description": "Html template with updated content after customization", "type": "string"}}, "required": ["html_template"]}\n```'

In [54]:
html_customiz_chain = prompt | llm  | parser 

In [55]:
website_content = {
    "What is the desired brand name- logo text for the website header and footer?": "TechNova",
    "What wording should be used for the main heading (H1) on the hero section of the landing page?": "Revolutionizing Your Digital Experience",
    "What descriptive text should accompany the main heading in the hero section to explain the business offering?": "Empower your business with cutting-edge solutions designed to streamline operations and maximize efficiency.",
    "What call-to-action text should be used on the primary button in the hero section?": "Get Started Today",
    "What heading should be used for the 'Features' section to best describe the benefits offered?": "Why Choose TechNova?",
    "What wording should be used for each of the individual feature titles- for example- Smart Automation- Advanced Analytics- Enterprise Security?": [
        "Smart Automation",
        "Advanced Analytics",
        "Enterprise Security",
        "Seamless Integration"
    ],
    "What wording should be used for the 'Testimonials' section heading to convey customer satisfaction?": "What Our Clients Say",
    "What is the desired wording for the call-to-action heading and supporting text in the email collection section?": {
        "heading": "Stay Updated with Our Latest Innovations",
        "text": "Subscribe to our newsletter for exclusive insights, updates, and special offers."
    },
    "What placeholder text should be used in the email input field of the email collection form?": "Abdelrahman.m2922@gmail.com",
    "What wording should be used for the navigation links (e.g.- Features- Testimonials- Pricing- Contact)?": [
        "Features",
        "Testimonials",
        "Pricing",
        "Contact"
    ]
}

website_content


{'What is the desired brand name- logo text for the website header and footer?': 'TechNova',
 'What wording should be used for the main heading (H1) on the hero section of the landing page?': 'Revolutionizing Your Digital Experience',
 'What descriptive text should accompany the main heading in the hero section to explain the business offering?': 'Empower your business with cutting-edge solutions designed to streamline operations and maximize efficiency.',
 'What call-to-action text should be used on the primary button in the hero section?': 'Get Started Today',
 "What heading should be used for the 'Features' section to best describe the benefits offered?": 'Why Choose TechNova?',
 'What wording should be used for each of the individual feature titles- for example- Smart Automation- Advanced Analytics- Enterprise Security?': ['Smart Automation',
  'Advanced Analytics',
  'Enterprise Security',
  'Seamless Integration'],
 "What wording should be used for the 'Testimonials' section head

In [56]:
response = html_customiz_chain.invoke({
    'html_template' : html,
    'Q_A_list' : website_content
    })

In [63]:
import json
json.dumps(response)

'{"html_template": "<body>\\n  <!-- Header & Navigation -->\\n  <header>\\n    <div class=\\"container\\">\\n      <nav>\\n        <a href=\\"#\\" class=\\"logo\\">TechNova</a>\\n        <div class=\\"menu-toggle\\" id=\\"menuToggle\\">\\n          <span class=\\"bar\\"></span>\\n          <span class=\\"bar\\"></span>\\n          <span class=\\"bar\\"></span>\\n        </div>\\n        <div class=\\"nav-links\\" id=\\"navLinks\\">\\n          <a href=\\"#features\\">Features</a>\\n          <a href=\\"#testimonials\\">Testimonials</a>\\n          <a href=\\"#pricing\\">Pricing</a>\\n          <a href=\\"#contact\\">Contact</a>\\n        </div>\\n        <button class=\\"btn\\">Get Started</button>\\n      </nav>\\n    </div>\\n  </header>\\n\\n  <!-- Hero Section -->\\n  <section class=\\"hero\\">\\n    <div class=\\"container\\">\\n      <div class=\\"hero-content\\">\\n        <div class=\\"hero-text\\">\\n          <h1>Revolutionizing Your Digital Experience</h1>\\n          <p>Emp

In [58]:
print(response['html_template'])
    

<body>
  <!-- Header & Navigation -->
  <header>
    <div class="container">
      <nav>
        <a href="#" class="logo">TechNova</a>
        <div class="menu-toggle" id="menuToggle">
          <span class="bar"></span>
          <span class="bar"></span>
          <span class="bar"></span>
        </div>
        <div class="nav-links" id="navLinks">
          <a href="#features">Features</a>
          <a href="#testimonials">Testimonials</a>
          <a href="#pricing">Pricing</a>
          <a href="#contact">Contact</a>
        </div>
        <button class="btn">Get Started</button>
      </nav>
    </div>
  </header>

  <!-- Hero Section -->
  <section class="hero">
    <div class="container">
      <div class="hero-content">
        <div class="hero-text">
          <h1>Revolutionizing Your Digital Experience</h1>
          <p>Empower your business with cutting-edge solutions designed to streamline operations and maximize efficiency.</p>
          <button class="btn">Get Started 